In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
df_train_labels = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

In [ ]:
df_averaged = df_train.groupby(['sequence','subject']).mean().reset_index()
df_averaged = df_averaged.drop(labels = ['sequence','step'],axis=1)
df_averaged.columns = [''.join(str(col)).strip()+'_avg' for col in df_averaged.columns.values]
df_averaged.head()


df_std = df_train.groupby(['sequence','subject']).std().reset_index()
df_std = df_std.drop(labels = ['sequence','step'],axis=1)
df_std.columns = [''.join(str(col)).strip()+'_std' for col in df_std.columns.values]
df_std.head()


df_max = df_train.groupby(['sequence','subject']).max().reset_index()
df_max = df_max.drop(labels = ['sequence','step'],axis=1)
df_max.columns = [''.join(str(col)).strip()+'_max' for col in df_max.columns.values]
df_max.head()

df_min = df_train.groupby(['sequence','subject']).min().reset_index()
df_min = df_min.drop(labels = ['sequence','step'],axis=1)
df_min.columns = [''.join(str(col)).strip()+'_min' for col in df_min.columns.values]
df_min.head()


df_fin = pd.concat([df_averaged,df_std,df_min,df_max],axis=1)
df_fin = df_fin.drop(labels = ['subject_std','subject_min','subject_max'],axis=1)
df_fin.head()



In [ ]:
target = df_train_labels['state']
input_matrix = df_fin
from sklearn.impute import SimpleImputer
imp=SimpleImputer( missing_values=np.nan,strategy='mean',fill_value=None,verbose=0,copy=True )
input_matrix=imp.fit_transform(input_matrix)
from sklearn import preprocessing
input_matrix= preprocessing.StandardScaler().fit(input_matrix).transform(input_matrix)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
#Split Data for training and testing
X_train, X_test, y_train, y_test = train_test_split(input_matrix, target,test_size=0.33)

from sklearn import svm

#Finding best kernel
knls = ['linear', 'rbf', 'sigmoid']
acc_svm = []
for kernel in knls:
    clf_krnl = svm.SVC(kernel=kernel,gamma="auto").fit(X_train, y_train)
    svm_predict = clf_krnl.predict(X_test)
    acc_svm.append(metrics.accuracy_score(y_test, svm_predict)) 
    
print("Maximum accuracy:",max(acc_svm),"with Kernel =",knls[acc_svm.index(max(acc_svm))])

In [ ]:
#Training the model with best kernel 
kernel="rbf"
clf_krnl = svm.SVC(kernel=kernel,gamma="auto").fit(X_train, y_train)

In [ ]:
df_averaged = df_test.groupby(['sequence','subject']).mean().reset_index()
df_averaged = df_averaged.drop(labels = ['sequence','step'],axis=1)
df_averaged.columns = [''.join(str(col)).strip()+'_avg' for col in df_averaged.columns.values]
df_averaged.head()


df_std = df_test.groupby(['sequence','subject']).std().reset_index()
df_std = df_std.drop(labels = ['sequence','step'],axis=1)
df_std.columns = [''.join(str(col)).strip()+'_std' for col in df_std.columns.values]
df_std.head()


df_max = df_test.groupby(['sequence','subject']).max().reset_index()
df_max = df_max.drop(labels = ['sequence','step'],axis=1)
df_max.columns = [''.join(str(col)).strip()+'_max' for col in df_max.columns.values]
df_max.head()

df_min = df_test.groupby(['sequence','subject']).min().reset_index()
df_min = df_min.drop(labels = ['sequence','step'],axis=1)
df_min.columns = [''.join(str(col)).strip()+'_min' for col in df_min.columns.values]
df_min.head()


df_fin = pd.concat([df_averaged,df_std,df_min,df_max],axis=1)
df_fin = df_fin.drop(labels = ['subject_std','subject_min','subject_max'],axis=1)
df_fin.head()

input_test_matrix = df_fin
from sklearn.impute import SimpleImputer
imp=SimpleImputer( missing_values=np.nan,strategy='mean',fill_value=None,verbose=0,copy=True )
input_test_matrix=imp.fit_transform(input_test_matrix)
from sklearn import preprocessing
input_test_matrix= preprocessing.StandardScaler().fit(input_test_matrix).transform(input_test_matrix)



In [ ]:
yhat_svm = clf_krnl.predict(input_test_matrix)

In [ ]:
import csv
with open('submission.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['sequence', 'state'])
    for i,v in enumerate(yhat_svm):
        csvwriter.writerow([i+25968,str(v)])